In [13]:
import os
from pathlib import Path
import subprocess


In [16]:
files = [x.as_posix() for x in Path().rglob('*.*') if not x.as_posix().endswith('.fna')]

In [4]:
from pathlib import Path
Path.home().as_posix()

'/Users/aaron'

In [29]:
import os
import pandas as pd
from pathlib import Path
import sys

info = '''
# Mycobacterium avium strain {reference}
{reference}.genome : Mycobacterium avium strain {reference}
    {reference}.chromosomes : {chromosomes}
    {reference}.{chromosome_0}.codonTable : Bacterial_and_Plant_Plastid
'''.strip('\n').split('\n')

def get_chromosomes(fna):
    with open(fna, 'r') as f:
        chromosomes = ', '.join([header.split('|')[-1:][0].strip('\n') for header in f.readlines() if '>' in header])
    return(chromosomes)

def edit_config(reference):
    config = '{}/snpEff/snpEff.config'.format(Path.home().as_posix())
    target = '# Mycobacterium avium strain {}'.format(reference)
    f = open(config, 'r')
    lines = f.readlines()
    if not target in lines:
        n = 0
        dictionary = {'part_1': [], 'part_3': []}
        for l in lines:
            n += 1
            if n < 235:
                dictionary['part_1'].append(l)
            else:
                dictionary['part_3'].append(l)
        fna = 'prokka/{reference}/{reference}.fna'.format(reference=reference)
        chromosomes = get_chromosomes(fna)
        chromosome_0 = chromosomes.split(', ')[0]
        part_2 = '\n'.join(info).format(reference=reference, chromosomes=chromosomes, chromosome_0=chromosome_0) + '\n'
        lines = dictionary['part_1'] + [part_2] + dictionary['part_3']
        with open(config, 'w') as f:
            [f.write(line) for line in lines]

def snpEff_build(reference):
    out_dir = '{}/snpEff/data/{}'.format(Path.home().as_posix(), reference)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    cds = 'cp prokka/{ref}/{ref}.ffn {out_dir}/cds.fa'.format(ref=reference, out_dir=out_dir)
    faa = 'cp prokka/{ref}/{ref}.faa {out_dir}/protein.faa'.format(ref=reference, out_dir=out_dir)
    gbk = 'cp prokka/{ref}/{ref}.gbk {out_dir}/genes.gbk'.format(ref=reference, out_dir=out_dir)
    snpEff_build = 'java -jar {}/snpEff/snpEff.jar build -genbank -v {}'.format(Path.home().as_posix(), reference)
    cmd = '{} && {} && {} && {}'.format(cds, faa, gbk, snpEff_build)
    return(cmd)

def snpEff_main(inp_vcf, out_dir, reference):
    if not inp_vcf.endswith('.gz'):
        cmd_1 = 'sed -i \'s/gnl|X|//g\' {}'.format(inp_vcf)
    else:
        cmd_1 = 'zcat < {} | sed \'s/gnl|X|//g\' > {}'.format(inp_vcf, inp_vcf.rstrip('.gz'))
        inp_vcf = inp_vcf.rstrip('.gz')
    out_vcf = '{}/{}'.format(out_dir, os.path.basename(inp_vcf).replace('.vcf', '.snpeff.vcf'))
    cmd_2 = 'java -jar {}/snpEff/snpEff.jar {} {} > {}'.format(Path.home().as_posix(), reference, inp_vcf, out_vcf)
    cmd = '{} && {}'.format(cmd_1, cmd_2)
    return(cmd)

def run_snpEff(dictionary, extension='vcf'):
    cmds = []
    patient = dictionary['patient']
    reference = dictionary['reference']
    ## make the database
    edit_config(reference)
    cmds.append(snpEff_build(reference))
    ## run snpEff
    queries = dictionary['query']
    for query in queries:
        out_dir = 'within_host_evolution/{}'.format(patient)
        inp_vcf = '{}/{}.{}'.format(out_dir, query, extension)
        cmds.append(snpEff_main(inp_vcf, out_dir, reference))
    cmd = ' && '.join(cmds)
    return(cmd)

def get_ref(patient, df):
    df = df[df['patient'] == patient]
    reference = df.nsmallest(1, 'time_from_diagnosis').iloc[0, 1]
    query = df[df['isolate'] != reference]['isolate'].to_list()
    output = {'patient': patient, 'reference': reference, 'query': query}
    return(output)

def list_patients(df):
    df = df[['patient', 'multiple_carriage']].drop_duplicates()
    patients = df[df['multiple_carriage']==False]['patient'].to_list()
    return(patients)

In [30]:
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

dat = pd.read_csv('data/metadata.tsv', sep='\t')
extension = 'lofreq.vcf.gz'
patients = list_patients(dat)
cmds = []
for patient in patients:
    dictionary = get_ref(patient, dat)
    cmd = run_snpEff(dictionary, extension)
    cmds.append(cmd)
with open('run_within_host_snpeff.sh', 'w') as f:
    for cmd in cmds:
        if len(cmd.replace(' ', '')) > 0:
            f.write(cmd + '\n')


In [ ]:
metadata = pd.read_csv('data/metadata.tsv', sep='\t')


In [53]:
import os
from pathlib import Path
import subprocess

os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

reads_r1 = ['reads/processed/{}_R1_001.qcd.fastq.gz'.format(x) for x in metadata['isolate'].to_list()]
reads_r2 = ['reads/processed/{}_R2_001.qcd.fastq.gz'.format(x) for x in metadata['isolate'].to_list()]
reads = reads_r1 + reads_r2

files = [x.as_posix() for x in Path().rglob('reads/processed/*.qcd.fastq.gz') if not x.as_posix() in reads]
for f in files:
    cmd = 'rm {}'.format(f)
    subprocess.run(cmd, shell=True)

In [62]:
import pandas as pd

df = pd.read_csv('data/metadata.tsv', sep='\t')
isolates = df[df['study'] == 'Wetzstein']['isolate'].to_list()

spades = 'spades.py -1 reads/processed/{isolate}_R1_001.qcd.fastq.gz -2 reads/processed/{isolate}_R2_001.qcd.fastq.gz -o spades/{isolate} --isolate -t 12 -m 32'

assemblies = ['spades/{}/contigs.fasta'.format(isolate) for isolate in isolates]
with open('run_spades.sh', 'w') as f:
    for isolate in isolates:
        assembly = 'spades/{}/contigs.fasta'.format(isolate)
        if not os.path.exists(assembly):
            cmd = spades.format(isolate=isolate)
            f.write(cmd + '\n')

In [64]:
n = 0
for isolate in isolates:
    assembly = 'assemblies/{}.contigs.fasta.gz'
    if not os.path.exists(assembly):
        n += 1

In [ ]:
import glob
assemblies = glob.glob('assemblies/*.contigs.fasta.gz')
n = 0
for assembly in assemblies:
    isolate = assembly.split('/')[1].split('.')[0]
    if not isolate in isolates:
        print(isolate)

['ERR12861894', 'ERR12861904', 'ERR12861907', 'ERR12861559', 'ERR12861671', 'ERR12861672', 'ERR12861491', 'ERR12861493', 'ERR12861513', 'ERR12861557', 'ERR12861569', 'ERR12861584', 'ERR12861589', 'ERR12861603', 'ERR12861607', 'ERR12861610', 'ERR12861619', 'ERR12861643', 'ERR12861681', 'ERR12861682', 'ERR12861544', 'ERR12861547', 'ERR12861579', 'ERR12861581', 'ERR12861457', 'ERR12861560', 'ERR12861568', 'ERR12861572', 'ERR12861580', 'ERR12861617', 'ERR12861631', 'ERR12861454', 'ERR12861472', 'ERR12861503', 'ERR12861510', 'ERR12861514', 'ERR12861515', 'ERR12861535', 'ERR12861556', 'ERR12861561', 'ERR12861582', 'ERR12861629', 'ERR12861664', 'ERR12861665', 'ERR12861595', 'ERR12861602', 'ERR12861605', 'ERR12861612', 'ERR12861620', 'ERR12861624', 'ERR12861630', 'ERR12861633', 'ERR12861634', 'ERR12861649', 'ERR12861684', 'ERR12861685', 'ERR12861551', 'ERR12861555', 'ERR12861636', 'ERR12861533', 'ERR12861548', 'ERR12861436', 'ERR12861437', 'ERR12861441', 'ERR12861448', 'ERR12861673', 'ERR12861

In [84]:
import glob
import subprocess

df = pd.read_csv('data/metadata.tsv', sep='\t')
isolates = df['isolate'].to_list()

assemblies = glob.glob('assemblies/*.contigs.fasta.gz')

for assembly in assemblies:
    isolate = assembly.split('/')[1].split('.')[0]
    if not isolate in isolates:
        cmd = 'rm {}'.format(assembly)
        subprocess.run(cmd, shell=True)

In [ ]:
import pandas as pd

df = pd.read_csv('data/metadata.tsv', sep='\t')
isolates = df['isolate'].to_list()

n = 0
for isolate in isolates:
    assembly = 'assemblies/{}.contigs.fasta.gz'.format(isolate)
    if os.path.exists(assembly):
        n += 1


185


In [94]:
import os
from pathlib import Path

fasta = [x.as_posix() for x in Path('spades').rglob('contigs.fasta')]
for f in fasta:
    gzip = 'gzip spades/{}/contigs.fasta'.format(f)
    subprocess.run(gzip, shell=True)

contigs = [x.as_posix() for x in Path('spades').rglob('contigs.fasta.gz')]
for contig in contigs:
    isolate = contig.split('/')[1]
    out = 'assemblies/{}.contigs.fasta.gz'.format(isolate)
    cp = 'cp {} {}'.format(contig, out)
    if not os.path.exists(out):
        subprocess.run(cp, shell=True)


In [ ]:
import pandas as pd

snippy_core = 'docker run -v $PWD:/data staphb/snippy snippy-core --ref databases/GCF_009741445.1/GCF_009741445.1_ASM974144v1_genomic.fna --prefix snippy/core.{} {}'
snippy_clean_full_aln = 'docker run -v $PWD:/data staphb/snippy snippy-clean_full_aln snippy/core.{x}.full.aln > snippy/gubbins.{x}.aln'
cmd = '{} && {}'.format(cmd_1, cmd_2)

df = pd.read_csv('data/fastbaps.tsv', sep='\t')
clusters = set(df['level_1'].to_list())

with open('run_snippy.sh', 'w') as f:
    for cluster in clusters:
        name = 'fastbaps_cluster_' + str(cluster).zfill(2)
        isolates = ' '.join(['snippy/{}'.format(x) for x in df[df['level_1'] == cluster]['isolate'].to_list() if not 'Reference' in x])
        cmd_1 = snippy_core.format(name, isolates)
        cmd_2 = snippy_clean_full_aln.format(x=name)
        cmd = '{} && {}'.format(cmd_1, cmd_2)
        f.write(cmd + '\n')


In [4]:
import glob
import os
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/gubbins/')
run_gubbins = 'run_gubbins.py --prefix {} {} --threads 12'
alignments = glob.glob('gubbins.*.aln')
with open('run_gubbins.sh', 'w') as f:
    for alignment in alignments:
        cluster = alignment.split('.')[1]
        if not os.path.exists('{}.filtered_polymorphic_sites.fasta'.format(cluster)):
            cmd = run_gubbins.format(cluster, alignment)
            f.write(cmd + '\n')


In [11]:
import glob
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')
assemblies = glob.glob('assemblies/*')
with open('ska/file_list.txt', 'w') as f:
    for assembly in assemblies:
        name = os.path.basename(assembly).split('.')[0]
        f.write('{}\t{}\n'.format(name, assembly))

In [ ]:

ska_build = 'ska build -f assemblies.txt -k 31 -o ska/ska_index --threads 4'